# Hyperoptimization with optuna

## 1. Dataset

Example for dataset ESOL from Moleculenet.

In [1]:
import numpy as np
import tensorflow as tf

In [2]:
from kgcnn.data.datasets.ESOLDataset import ESOLDataset
dataset = ESOLDataset()
dataset.set_attributes()
dataset.map_list(method="set_edge_indices_reverse")

INFO:kgcnn.data.base:Reserved properties for `MemoryGraphList` will be removed in version 2.1.0. Please use set/get methods instead.
INFO:kgcnn.data.download:Checking and possibly downloading dataset with name ESOL
INFO:kgcnn.data.download:Dataset directory located at C:\Users\patri\.kgcnn\datasets
INFO:kgcnn.data.download:Dataset directory found. Done.
INFO:kgcnn.data.download:Dataset found. Done.
INFO:kgcnn.data.ESOL:Found SDF C:\Users\patri\.kgcnn\datasets\ESOL\delaney-processed.sdf of pre-computed structures.
INFO:kgcnn.data.ESOL: ... read molecules 0 from 1128
INFO:kgcnn.data.ESOL: ... read molecules 1000 from 1128
INFO:kgcnn.data.ESOL: ... read molecules 0 from 1128
INFO:kgcnn.data.ESOL: ... read molecules 1000 from 1128
INFO:kgcnn.mol.encoder:OneHotEncoder Symbol found ['O', 'C', 'N', 'S', 'Cl', 'P', 'F', 'I', 'Br']
INFO:kgcnn.mol.encoder:OneHotEncoder Hybridization found [rdkit.Chem.rdchem.HybridizationType.SP3, rdkit.Chem.rdchem.HybridizationType.SP, rdkit.Chem.rdchem.Hybridiz

## 2. Model

Pick DMPNN as example model.

In [3]:
from kgcnn.literature.DMPNN import make_model

In [4]:
model_config = {
    "name": "DMPNN",
    "inputs": [
        {"shape": [None, 41], "name": "node_attributes", "dtype": "float32", "ragged": True},
        {"shape": [None, 11], "name": "edge_attributes", "dtype": "float32", "ragged": True},
        {"shape": [None, 2], "name": "edge_indices", "dtype": "int64", "ragged": True},
        {"shape": [None, 1], "name": "edge_indices_reverse", "dtype": "int64", "ragged": True}
    ],
    "input_embedding": {"node": {"input_dim": 95, "output_dim": 64},
                        "edge": {"input_dim": 5, "output_dim": 64}},
    "pooling_args": {"pooling_method": "sum"},
    "edge_initialize": {"units": 128, "use_bias": True, "activation": "relu"},
    "edge_dense": {"units": 128, "use_bias": True, "activation": "linear"},
    "edge_activation": {"activation": "relu"},
    "node_dense": {"units": 128, "use_bias": True, "activation": "relu"},
    "verbose": 10, "depth": 5,
    "dropout": {"rate": 0.1},
    "output_embedding": "graph",
    "output_mlp": {
        "use_bias": [True, True, False], "units": [64, 32, 1],
        "activation": ["relu", "relu", "linear"]
    }
}
# Test making model
model = make_model(**model_config)

INFO:kgcnn.utils.models:Updated model kwargs:
INFO:kgcnn.utils.models:{'name': 'DMPNN', 'inputs': [{'shape': [None, 41], 'name': 'node_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 11], 'name': 'edge_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 2], 'name': 'edge_indices', 'dtype': 'int64', 'ragged': True}, {'shape': [None, 1], 'name': 'edge_indices_reverse', 'dtype': 'int64', 'ragged': True}], 'input_embedding': {'node': {'input_dim': 95, 'output_dim': 64}, 'edge': {'input_dim': 5, 'output_dim': 64}}, 'pooling_args': {'pooling_method': 'sum'}, 'edge_initialize': {'units': 128, 'use_bias': True, 'activation': 'relu'}, 'edge_dense': {'units': 128, 'use_bias': True, 'activation': 'linear'}, 'edge_activation': {'activation': 'relu'}, 'node_dense': {'units': 128, 'use_bias': True, 'activation': 'relu'}, 'verbose': 10, 'depth': 5, 'dropout': {'rate': 0.1}, 'output_embedding': 'graph', 'output_to_tensor': True, 'output_mlp': {'use_bias': [True, True,

In [5]:
dataset.clean(model_config["inputs"])

INFO:kgcnn.data.ESOL:Property edge_attributes is an empty list for graph 934.
INFO:kgcnn.data.ESOL:Property edge_indices is an empty list for graph 934.
INFO:kgcnn.data.ESOL:Property edge_indices_reverse is an empty list for graph 934.


array([934])

In [6]:
labels = np.expand_dims(dataset.get("graph_labels"), axis=-1)
labels.shape

(1127, 1)

## 3. Optuna

Loading optuna for hyperoptimization example. Example is modified from optuna [github](https://github.com/optuna/optuna-examples/blob/main/keras/keras_simple.py>) or web page https://optuna.org/#code_examples.

For this example we neglect target standardization or more metric options.

In [7]:
import warnings
import optuna
from copy import deepcopy
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from keras.backend import clear_session

In [8]:
BATCHSIZE = 128
EPOCHS = 100

In [9]:
# Define objective function
def objective(trial):
    # Clear clutter from previous Keras session graphs.
    clear_session()

    train_index, test_index = train_test_split(np.arange(len(dataset)), test_size=0.25, random_state=42)
    x_train, y_train = dataset[train_index].tensor(model_config["inputs"]), labels[train_index]
    x_valid, y_valid = dataset[test_index].tensor(model_config["inputs"]), labels[test_index]
    
    hyper_trial = deepcopy(model_config)
    hyper_trial["depth"] = trial.suggest_int("depth", 1, 6, 2)
    hyper_trial["pooling_args"]["pooling_method"] = trial.suggest_categorical("pooling_method", ["sum", "mean"])
    units = trial.suggest_int("units", 32, 256, 32)
    hyper_trial["node_dense"]["units"] = units
    hyper_trial["edge_dense"]["units"] = units
    hyper_trial["edge_initialize"]["units"] = units
    # print(hyper_trial)
    
    model = make_model(**hyper_trial)
    
    # We compile our model
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    model.compile(
        loss="mean_squared_error",
        optimizer=Adam(learning_rate=learning_rate),
        metrics=["mean_absolute_error"],
    )

    model.fit(
        x_train,
        y_train,
        validation_data=(x_valid, y_valid),
        shuffle=True,
        batch_size=BATCHSIZE,
        epochs=EPOCHS,
        verbose=False,
    )

    # Evaluate the model accuracy on the validation set.
    score = model.evaluate(x_valid, y_valid, verbose=0)
    return score[1]

### Optimization

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=100, timeout=600)

[I 2022-08-25 11:51:58,808] A new study created in memory with name: no-name-965b2b3e-dd7d-4955-a0f7-f337a3e3576e
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\optuna\distributions.py:560: UserWarning: The distribution is specified by [1, 6] and step=2, but the range is not divisible by `step`. It will be replaced by [1, 5].
  warnings.warn(
INFO:kgcnn.utils.models:Updated model kwargs:
INFO:kgcnn.utils.models:{'name': 'DMPNN', 'inputs': [{'shape': [None, 41], 'name': 'node_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 11], 'name': 'edge_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 2], 'name': 'edge_indices', 'dtype': 'int64', 'ragged': True}, {'shape': [None, 1], 'name': 'edge_indices_reverse', 'dtype': 'int64', 'ragged': True}], 'input_embedding': {'node': {'input_dim': 95, 'output_dim': 64}, 'edge': {'input_dim': 5, 'output_dim': 64}}, 'pooling_args': {'pooling_method': 'mean'}, 'edge_initialize': {'units': 160, 'use_bias'

[I 2022-08-25 11:52:29,861] Trial 1 finished with value: 1.3601354360580444 and parameters: {'depth': 3, 'pooling_method': 'sum', 'units': 32, 'learning_rate': 1.2188337506301997e-05}. Best is trial 0 with value: 0.5669612288475037.
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\optuna\distributions.py:560: UserWarning: The distribution is specified by [1, 6] and step=2, but the range is not divisible by `step`. It will be replaced by [1, 5].
  warnings.warn(
INFO:kgcnn.utils.models:Updated model kwargs:
INFO:kgcnn.utils.models:{'name': 'DMPNN', 'inputs': [{'shape': [None, 41], 'name': 'node_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 11], 'name': 'edge_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 2], 'name': 'edge_indices', 'dtype': 'int64', 'ragged': True}, {'shape': [None, 1], 'name': 'edge_indices_reverse', 'dtype': 'int64', 'ragged': True}], 'input_embedding': {'node': {'input_dim': 95, 'output_dim': 64}, 'edge': {'inpu

[I 2022-08-25 11:52:52,908] Trial 2 finished with value: 0.8592892289161682 and parameters: {'depth': 5, 'pooling_method': 'mean', 'units': 192, 'learning_rate': 3.3221645095785274e-05}. Best is trial 0 with value: 0.5669612288475037.
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\optuna\distributions.py:560: UserWarning: The distribution is specified by [1, 6] and step=2, but the range is not divisible by `step`. It will be replaced by [1, 5].
  warnings.warn(
INFO:kgcnn.utils.models:Updated model kwargs:
INFO:kgcnn.utils.models:{'name': 'DMPNN', 'inputs': [{'shape': [None, 41], 'name': 'node_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 11], 'name': 'edge_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 2], 'name': 'edge_indices', 'dtype': 'int64', 'ragged': True}, {'shape': [None, 1], 'name': 'edge_indices_reverse', 'dtype': 'int64', 'ragged': True}], 'input_embedding': {'node': {'input_dim': 95, 'output_dim': 64}, 'edge': {'in

[I 2022-08-25 11:53:12,594] Trial 3 finished with value: 0.5624402761459351 and parameters: {'depth': 5, 'pooling_method': 'mean', 'units': 64, 'learning_rate': 0.006586272794336998}. Best is trial 3 with value: 0.5624402761459351.
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\optuna\distributions.py:560: UserWarning: The distribution is specified by [1, 6] and step=2, but the range is not divisible by `step`. It will be replaced by [1, 5].
  warnings.warn(
INFO:kgcnn.utils.models:Updated model kwargs:
INFO:kgcnn.utils.models:{'name': 'DMPNN', 'inputs': [{'shape': [None, 41], 'name': 'node_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 11], 'name': 'edge_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 2], 'name': 'edge_indices', 'dtype': 'int64', 'ragged': True}, {'shape': [None, 1], 'name': 'edge_indices_reverse', 'dtype': 'int64', 'ragged': True}], 'input_embedding': {'node': {'input_dim': 95, 'output_dim': 64}, 'edge': {'input

C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/pooling_local_edges_3/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/pooling_local_edges_3/Reshape:0", shape=(None, 128), dtype=float32), dense_shape=Tensor("gradient_tape/model/pooling_local_edges_3/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/dmpnnp_pooling_edges_directed_2/gather_nodes_outgoing_3/Reshape_1:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/model/dmpnnp_pooling_edges_directed_2/gather_nodes_outgoing_3/Reshape:0", shape=(None, 12

...

[I 2022-08-25 11:58:43,197] Trial 23 finished with value: 0.4693397879600525 and parameters: {'depth': 3, 'pooling_method': 'sum', 'units': 64, 'learning_rate': 0.003223930632882858}. Best is trial 23 with value: 0.4693397879600525.
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\optuna\distributions.py:560: UserWarning: The distribution is specified by [1, 6] and step=2, but the range is not divisible by `step`. It will be replaced by [1, 5].
  warnings.warn(
INFO:kgcnn.utils.models:Updated model kwargs:
INFO:kgcnn.utils.models:{'name': 'DMPNN', 'inputs': [{'shape': [None, 41], 'name': 'node_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 11], 'name': 'edge_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 2], 'name': 'edge_indices', 'dtype': 'int64', 'ragged': True}, {'shape': [None, 1], 'name': 'edge_indices_reverse', 'dtype': 'int64', 'ragged': True}], 'input_embedding': {'node': {'input_dim': 95, 'output_dim': 64}, 'edge': {'inpu

[I 2022-08-25 11:59:12,076] Trial 25 finished with value: 0.6305660009384155 and parameters: {'depth': 3, 'pooling_method': 'sum', 'units': 32, 'learning_rate': 0.035275016950034446}. Best is trial 23 with value: 0.4693397879600525.
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\optuna\distributions.py:560: UserWarning: The distribution is specified by [1, 6] and step=2, but the range is not divisible by `step`. It will be replaced by [1, 5].
  warnings.warn(
INFO:kgcnn.utils.models:Updated model kwargs:
INFO:kgcnn.utils.models:{'name': 'DMPNN', 'inputs': [{'shape': [None, 41], 'name': 'node_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 11], 'name': 'edge_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 2], 'name': 'edge_indices', 'dtype': 'int64', 'ragged': True}, {'shape': [None, 1], 'name': 'edge_indices_reverse', 'dtype': 'int64', 'ragged': True}], 'input_embedding': {'node': {'input_dim': 95, 'output_dim': 64}, 'edge': {'inpu

[I 2022-08-25 11:59:31,916] Trial 26 finished with value: 0.497846782207489 and parameters: {'depth': 5, 'pooling_method': 'sum', 'units': 64, 'learning_rate': 0.00292549205204378}. Best is trial 23 with value: 0.4693397879600525.
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\optuna\distributions.py:560: UserWarning: The distribution is specified by [1, 6] and step=2, but the range is not divisible by `step`. It will be replaced by [1, 5].
  warnings.warn(
INFO:kgcnn.utils.models:Updated model kwargs:
INFO:kgcnn.utils.models:{'name': 'DMPNN', 'inputs': [{'shape': [None, 41], 'name': 'node_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 11], 'name': 'edge_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 2], 'name': 'edge_indices', 'dtype': 'int64', 'ragged': True}, {'shape': [None, 1], 'name': 'edge_indices_reverse', 'dtype': 'int64', 'ragged': True}], 'input_embedding': {'node': {'input_dim': 95, 'output_dim': 64}, 'edge': {'input_

C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/pooling_local_edges_1/Reshape_1:0", shape=(None,), dtype=int32), values=Tensor("gradient_tape/model/pooling_local_edges_1/Reshape:0", shape=(None, 32), dtype=float32), dense_shape=Tensor("gradient_tape/model/pooling_local_edges_1/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/dmpnnp_pooling_edges_directed/gather_nodes_outgoing_1/Reshape_1:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/model/dmpnnp_pooling_edges_directed/gather_nodes_outgoing_1/Reshape:0", shape=(None, 32), dt

[I 2022-08-25 12:00:19,571] Trial 29 finished with value: 0.5417131781578064 and parameters: {'depth': 3, 'pooling_method': 'mean', 'units': 160, 'learning_rate': 0.0008087168469543698}. Best is trial 23 with value: 0.4693397879600525.
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\optuna\distributions.py:560: UserWarning: The distribution is specified by [1, 6] and step=2, but the range is not divisible by `step`. It will be replaced by [1, 5].
  warnings.warn(
INFO:kgcnn.utils.models:Updated model kwargs:
INFO:kgcnn.utils.models:{'name': 'DMPNN', 'inputs': [{'shape': [None, 41], 'name': 'node_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 11], 'name': 'edge_attributes', 'dtype': 'float32', 'ragged': True}, {'shape': [None, 2], 'name': 'edge_indices', 'dtype': 'int64', 'ragged': True}, {'shape': [None, 1], 'name': 'edge_indices_reverse', 'dtype': 'int64', 'ragged': True}], 'input_embedding': {'node': {'input_dim': 95, 'output_dim': 64}, 'edge': {'i

In [ ]:
print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))